In [9]:
import random
import sys
import time

decision = input('''Silahkan pilih strategi algoritma untuk penyelesaian masalah 0/1 Knapsack
1: Backtracking (Depth First Search)
2: Branch and Bound (Breadth First Search)
3: Branch and Bound (Best First Search)
4: Keluar
Pilihan: ''')


def decider(x):
    return {
        '1': 'DepthFirst',
        '2': 'BreadthFirst',
        '3': 'BestFirst',
    }[x]


if decision == '4':
    sys.exit()

size = input('''Berapa barang (elektronik) random yang akan disediakan?
a. 10
b. 15
c. 25
Pilihan? ''')

weight = []  # berat setiap barang
value = []  # harga (keuntungan) setiap barang
n = len(weight)  # jumlah barang yang tersedia
sum_w = sum(weight)  # jumlah berat barang yang tersediia
capacity = 0  # kapasitas ransel (knapsack)
density = []  # harga per unit pada setiap barang

if size == 'a':
    weight = [70, 50, 48, 60, 32, 22, 50, 25, 10, 4]
    value = [192, 162, 122, 110, 96, 82, 70, 63, 25, 5]
    n = len(weight)
    sum_w = sum(weight)
    capacity = 200
    for i, j in enumerate(weight):
        density.append(round(value[i] / j, 3))
elif size == 'b':
    weight = [82, 70, 25, 55, 32, 32, 38, 28, 30, 30, 65, 10, 25, 10, 1]
    value = [208, 180, 158, 130, 118, 101, 100, 90, 77, 73, 66, 58, 50, 10, 1]
    n = len(weight)
    sum_w = sum(weight)
    capacity = 300
    for i, j in enumerate(weight):
        density.append(round(value[i] / j, 3))
elif size == 'c':
    weight = [80, 85, 72, 66, 55, 50, 40, 50, 22, 30, 40, 35, 25, 30, 50, 45, 60, 20, 20, 30, 20, 15, 10, 4,
              2]
    value = [220, 198, 185, 165, 160, 155, 125, 120, 115, 105, 100, 98, 95, 88, 80, 75, 72, 69, 65, 60, 56, 30, 15, 8,
             3]
    n = len(weight)
    sum_w = sum(weight)
    capacity = 500
    for i, j in enumerate(weight):
        density.append(round(value[i] / j, 3))
# {selain a, b, c}. Antara 25 sampai 50 (berat dan harga acak)
# else:
#     n = 5*random.randint(5, 10)
#     for i in range(1, n):
#         weight.append(random.randint(1, 100))
#         value.append(random.randint(1, 250))
#         sum_items = sum(weight)
#         C = 20 * n
#         for i, j in enumerate(weight):
#             density.append(round(value[i] / j, 3))

print('kapasitas = {}\nberat = {}\nkeuntungan = {}\njumlah berat = {}\nkepadatan = {}'.format(capacity, weight, value,
                                                                                              sum_w, density))

traversal = decider(decision)
start_time = time.time()


# Membuat kelas Node (simpul) untuk menentukan atributnya
class Node:
    def __init__(self, level, value, weight):
        self.level = level
        self.value = value
        self.weight = weight
        self.selected = []
        self.opt = []
        self.side = []


# Linear Relaxation Solution/Solusi Relaksasi Linier (Sebuah solusi optimum untuk menemukan keuntungan maksimum)
def lrs(node):
    if node.weight >= capacity:
        return 0
    else:
        opt_val = node.value  # Nilai optimum (keuntungan maksimum)
        total_weight = node.weight
        order = sorted(range(len(density)), key=lambda i: density[i], reverse=True)
        for rem in range(node.level + 1):
            order.remove(rem)
        for j in order:
            if total_weight + weight[j] <= capacity:
                total_weight = total_weight + weight[j]
                opt_val = opt_val + value[j]
            else:
                opt_val = opt_val + round((capacity - total_weight) * density[j], 3)
                break
    return opt_val


class Queue:  # kami membutuhkan kendali atas apa yang terjadi dalam queue (antrian) node untuk diselidiki
    def __init__(self):  # kelas queue ini memiliki dua atribut:
        self.q = []  # q untuk memiliki akses ke node yang dilalui
        self.length = 0  # panjang antrian untuk melihat apakah antrian kosong atau tidak

    def add(self, node):  # fungsi add untuk menambahkan node yang dilalui ke antrian
        if traversal == 'BestFirst':  # jika memilih strategi BestFirst,
            # maka algoritma akan menempatkan node yang ditambahkan secara berurutan berdasarkan tujuan optimis mereka
            i = 0  # kami set nilai default i ke nol karena antrian bisa kosong
            for i in range(
                    self.length):  # kami menemukan tempat di mana node baru tidak mengganggu urutan menaik dari node berdasarkan nilai optimisnya
                if self.q[i].opt > node.opt:
                    break
            self.q.insert(i, node)
            self.length += 1

        elif traversal == 'DepthFirst' or traversal == 'BreadthFirst':  # jika memilih strategi DepthFirst atau BreadthFirst, algoritma menambahkan node baru ke antrian secara kronologis
            self.q.insert(self.length, node)
            self.length += 1

    def remove(self):
        # Memilih cabang traversal BestFirst untuk melanjutkan eksplorasi node
        if traversal == 'BestFirst':
            if self.length != 0:  # strategi BestFirst cenderung memilih node paling optimis untuk memulai eksplorasi
                result = self.q.pop()
                self.length -= 1
                return result

        # Memilih cabang traversal BreadthFirst untuk melanjutkan eksplorasi node
        elif traversal == 'BreadthFirst':  # strategi BreadthFirst cenderung memilih node yang paling lama berada dalam antrian
            if self.length != 0:
                result = self.q.pop(0)
                self.length -= 1
                return result

        # Memilih cabang traversal DepthFirst untuk melanjutkan eksplorasi node
        elif traversal == 'DepthFirst':  # strategi DepthFirst cenderung berhasil eksplorasi yang cukup dalam hingga mencapai dead end (jalan buntu).
            # Algoritma tidak akan pergi ke cabang kanan sebelum menyelesaikan semua simpul di cabang kiri
            if self.length >= 2:
                if self.q[-1].side == 'right' and self.q[-2].side == 'left':
                    result = self.q.pop(self.length - 2)
                else:
                    result = self.q.pop()
                self.length -= 1
                return result
            elif self.length == 1:
                result = self.q.pop()
                self.length -= 1
                return result


# ------------------------------------------------------------------------------
traversal_queue = Queue()
prt = Node(-1, 0, 0)  # node induk (parent) berada di akar pohon
# argumen pertama: tingkat simpul di pohon, level ini mewakili item yang algoritma coba muat di ransel
# argumen kedua: harga (keuntungan) item
# argumen ketiga: berat item
# jelas harga dan bobotnya nol untuk akar karena indeks item dimulai dari nol
# karena dimulai dari akar, maka levelnya diatur ke -1
nodes = 1  # variabel ini akan menunjukkan kepada kami berapa banyak node yang kami miliki di pohon
Objective = 0  # variabel ini akan menunjukkan kepada kami keuntungan yang kami peroleh
prt.opt = lrs(prt)  # nilai optimis dari fungsi tujuan (setelah relaksasi linier)
print('\n Nilai optimum (keuntungan terbesar) dari Knapsack dengan batasan kapasitas: {}'.format(prt.opt))

traversal_queue.add(prt)

while traversal_queue.length != 0:
    prt = traversal_queue.remove()  # kami harus menghapus sebuah node dari antrian
    # berdasarkan strategi traversal dan menghasilkan beberapa node baru
    # di mana item berikutnya dapat dipilih atau tidak
    if prt.opt > Objective:  # jika nilai optimis dari simpul induk lebih baik dari
        # nilai objektif yang diperoleh sejauh ini, masuk akal untuk melanjutkan eksplorasi
        left_branch = Node(0, 0, 0)  # simpul baru (yang kiri di mana X=1) dihasilkan
        left_branch.level = prt.level + 1  # atribut node baru ditentukan
        left_branch.value = prt.value + value[left_branch.level]
        left_branch.weight = prt.weight + weight[left_branch.level]
        left_branch.selected = prt.selected.copy()  # kami harus menyalin urutan item yang dipilih sejauh ini
        left_branch.selected.append(left_branch.level)  # lalu menambahkan simpul saat ini ke pendahulunya
        nodes += 1  # karena kami telah menghasilkan simpul baru, kami menambah jumlah simpul

        left_branch.opt = lrs(left_branch)
        if left_branch.opt > Objective:
            left_branch.side = 'left'
            traversal_queue.add(left_branch)

        if left_branch.value > Objective and left_branch.weight <= capacity:  # jika nilai simpul baru membantu meningkatkan solusi selagi tidak melanggar batasan
            select = left_branch.selected
            Objective = left_branch.value

        right_branch = Node(left_branch.level, prt.value,
                            prt.weight)  # cabang sisi kanan dihasilkan di sini di mana variabel keputusan akan menjadi nol
        # maka nilai dan bobotnya akan sama dengan induknya
        right_branch.opt = lrs(right_branch)
        right_branch.selected = prt.selected.copy()
        nodes += 1
        if right_branch.opt > Objective:
            right_branch.side = 'right'
            traversal_queue.add(right_branch)
    else:
        print('Node ({}, {}, {}) merupakan jalan buntu (non-promising)'.format(prt.level, prt.value, prt.weight))

decision_vector = [0] * n
for i in range(n):
    if i in select:
        decision_vector[i] = 1
print('Vektor keputusan: {}'.format(decision_vector))
print('Item yang bisa diambil adalah item ke-{}\nAda {} dari {} item yang bisa dimasukkan ke knapsack'.format(select,
                                                                                                              len(select),
                                                                                                              n))
print('Keuntungan maksimum yang bisa diraih adalah {} dan jumlah simpul di pohon adalah {}'.format(Objective,
                                                                                                   nodes))
print('Waktu eksekusi adalah {} detik'.format(round(time.time() - start_time, 5)))


Silahkan pilih strategi algoritma untuk penyelesaian masalah 0/1 Knapsack
1: Backtracking (Depth First Search)
2: Branch and Bound (Breadth First Search)
3: Branch and Bound (Best First Search)
4: Keluar
Pilihan: 3
Berapa barang (elektronik) random yang akan disediakan?
a. 10
b. 15
c. 25
Pilihan? c
kapasitas = 500
berat = [80, 85, 72, 66, 55, 50, 40, 50, 22, 30, 40, 35, 25, 30, 50, 45, 60, 20, 20, 30, 20, 15, 10, 4, 2]
keuntungan = [220, 198, 185, 165, 160, 155, 125, 120, 115, 105, 100, 98, 95, 88, 80, 75, 72, 69, 65, 60, 56, 30, 15, 8, 3]
jumlah berat = 956
kepadatan = [2.75, 2.329, 2.569, 2.5, 2.909, 3.1, 3.125, 2.4, 5.227, 3.5, 2.5, 2.8, 3.8, 2.933, 1.6, 1.667, 1.2, 3.45, 3.25, 2.0, 2.8, 2.0, 1.5, 2.0, 1.5]

 Nilai optimum (keuntungan terbesar) dari Knapsack dengan batasan kapasitas: 1538.5
Node (15, 1326, 433) merupakan jalan buntu (non-promising)
Node (7, 780, 275) merupakan jalan buntu (non-promising)
Node (10, 1165, 389) merupakan jalan buntu (non-promising)
Node (10, 1145, 383)